In [ ]:
import pandas as pd
import json
import os

file_list = [f for f in os.listdir('../data') if f.endswith('.json')]

def format_data(file_list):
    results_list = []
    for i in file_list:
        with open(f"../data/{i}", 'r') as f:
            data = json.load(f)
           
            data_formatted = {"timestamp": data['timestamp'],
                            "prompt": data['prompt'],
                            "expected_output": data["expected_output"],
                            "expected_tool_names": data['expected_tool_names'],
                            "metrics": data['metrics']}

            results_list.append(data_formatted)
            
    return pd.DataFrame(results_list)

df = format_data(file_list)

In [ ]:
for i in range(3):
    df[f'metrics_{i}'] = df['metrics'].apply(lambda x: x[i])
    df[f'metrics_{i}_name'] = df[f'metrics_{i}'].apply(lambda x: x['name'])
    df[f'metrics_{i}_score'] = df[f'metrics_{i}'].apply(lambda x: x['score'])
    df[f'metrics_{i}_success'] = df[f'metrics_{i}'].apply(lambda x: x['success'])
    df[f'metrics_{i}_reason'] = df[f'metrics_{i}'].apply(lambda x: x['reason'])

clean_df = df[['timestamp', 'prompt','expected_output','expected_tool_names',
    'metrics_0_name', 'metrics_0_score', 'metrics_0_success', 'metrics_0_reason',
    'metrics_1_name', 'metrics_1_score', 'metrics_1_success', 'metrics_1_reason',
    'metrics_2_name', 'metrics_2_score', 'metrics_2_success', 'metrics_2_reason']]

clean_df['prompt_trunc'] = clean_df['prompt'].str[:30]
clean_df

In [ ]:
import altair as alt
metric_num = 2


charts = []
for metric_num in [0,1,2]:

    charts.append(alt.Chart(clean_df).mark_line().encode(
        x='timestamp',
        y=f'metrics_{metric_num}_score',
        color='prompt',
        facet=alt.Facet('prompt_trunc', columns=2)
    ).properties(title = f"Performance over Time, Metric: {clean_df[f'metrics_{metric_num}_name'].iloc[0]}", width=400)
    )

charts[0] & charts[1] & charts[2]

# Failed Tests Drilldown

In [ ]:
# clean_df.head()
from IPython.display import Markdown

for idx, row in clean_df.iterrows():

    if row['metrics_0_success'] is False:
        md = f"""
    Metric: {row['metrics_0_name']}
    Prompt: {row['prompt']} 
    
    Failure Reason: {row['metrics_0_reason']}
        """
        display(Markdown(md))
        
    if row['metrics_1_success'] is False:
        md = f"""
    Metric: {row['metrics_1_name']}
    Prompt: {row['prompt']} 
    
    Failure Reason: {row['metrics_1_reason']}
        """
        display(Markdown(md))
        
    if row['metrics_2_success'] is False:
        md = f"""
    Metric: {row['metrics_2_name']}
    Prompt: {row['prompt']} 
    
    Failure Reason: {row['metrics_2_reason']}
        """
        display(Markdown(md))